# Generatieve netwerken

Recurrent Neural Networks (RNNs) en hun varianten met gated cellen, zoals Long Short Term Memory Cells (LSTMs) en Gated Recurrent Units (GRUs), bieden een mechanisme voor taalmodellering, oftewel ze kunnen de volgorde van woorden leren en voorspellingen doen voor het volgende woord in een reeks. Dit stelt ons in staat om RNNs te gebruiken voor **generatieve taken**, zoals gewone tekstgeneratie, machinevertaling en zelfs beeldbeschrijving.

In de RNN-architectuur die we in de vorige eenheid hebben besproken, produceerde elke RNN-eenheid de volgende verborgen toestand als output. We kunnen echter ook een extra output toevoegen aan elke recurrente eenheid, waardoor we een **reeks** kunnen genereren (die even lang is als de oorspronkelijke reeks). Bovendien kunnen we RNN-eenheden gebruiken die bij elke stap geen invoer accepteren, maar alleen een initiële toestandsvector nemen en vervolgens een reeks outputs produceren.

In deze notebook richten we ons op eenvoudige generatieve modellen die ons helpen tekst te genereren. Voor de eenvoud bouwen we een **karakter-niveau netwerk**, dat tekst letter voor letter genereert. Tijdens de training moeten we een tekstcorpus nemen en deze splitsen in letterreeksen.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Het opbouwen van een karaktervocabulaire

Om een generatief netwerk op karakterniveau te bouwen, moeten we tekst opsplitsen in afzonderlijke karakters in plaats van woorden. Dit kan worden gedaan door een andere tokenizer te definiëren:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Laten we het voorbeeld bekijken van hoe we de tekst uit onze dataset kunnen coderen:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Een generatieve RNN trainen

De manier waarop we een RNN zullen trainen om tekst te genereren, is als volgt. Bij elke stap nemen we een reeks karakters van lengte `nchars` en vragen we het netwerk om het volgende uitvoerkarakter te genereren voor elk invoerkarakter:

![Afbeelding die een voorbeeld toont van een RNN die het woord 'HELLO' genereert.](../../../../../translated_images/nl/rnn-generate.56c54afb52f9781d.webp)

Afhankelijk van het specifieke scenario willen we mogelijk ook enkele speciale karakters opnemen, zoals *einde-van-sequentie* `<eos>`. In ons geval willen we het netwerk gewoon trainen voor eindeloze tekstgeneratie, dus stellen we de grootte van elke sequentie vast op `nchars` tokens. Bijgevolg zal elk trainingsvoorbeeld bestaan uit `nchars` invoer en `nchars` uitvoer (wat de invoersequentie is, verschoven met één symbool naar links). Een minibatch zal bestaan uit meerdere van dergelijke sequenties.

De manier waarop we minibatches zullen genereren, is door elke nieuwsberichttekst van lengte `l` te nemen en alle mogelijke invoer-uitvoercombinaties daaruit te genereren (er zullen `l-nchars` van dergelijke combinaties zijn). Deze vormen één minibatch, en de grootte van de minibatches zal bij elke trainingsstap verschillen.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Laten we het generatornetwerk definiëren. Het kan gebaseerd zijn op elke recurrente cel die we in de vorige eenheid hebben besproken (eenvoudig, LSTM of GRU). In ons voorbeeld zullen we LSTM gebruiken.

Omdat het netwerk tekens als invoer neemt en de vocabulairegrootte vrij klein is, hebben we geen embeddinglaag nodig; een one-hot-gecodeerde invoer kan direct naar de LSTM-cel gaan. Echter, omdat we tekennummers als invoer doorgeven, moeten we ze one-hot-encoderen voordat we ze naar de LSTM sturen. Dit wordt gedaan door de functie `one_hot` aan te roepen tijdens de `forward`-pass. De uitvoerencoder zal een lineaire laag zijn die de verborgen toestand omzet in een one-hot-gecodeerde uitvoer.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Tijdens de training willen we in staat zijn om gegenereerde tekst te bemonsteren. Om dit te doen, definiëren we de functie `generate`, die een uitvoerstring van lengte `size` zal produceren, beginnend met de initiële string `start`.

De werkwijze is als volgt. Eerst geven we de volledige startstring door het netwerk en nemen we de uitvoerstatus `s` en het volgende voorspelde teken `out`. Aangezien `out` one-hot gecodeerd is, nemen we `argmax` om de index van het teken `nc` in de woordenschat te verkrijgen, en gebruiken we `itos` om het daadwerkelijke teken te achterhalen en voegen dit toe aan de resulterende lijst van tekens `chars`. Dit proces van het genereren van één teken wordt `size` keer herhaald om het vereiste aantal tekens te genereren.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Laten we beginnen met de training! De trainingslus is bijna hetzelfde als in al onze eerdere voorbeelden, maar in plaats van nauwkeurigheid printen we elke 1000 epochs een willekeurig gegenereerde tekst.

Er moet speciale aandacht worden besteed aan de manier waarop we verlies berekenen. We moeten het verlies berekenen op basis van de één-hot-gecodeerde output `out` en de verwachte tekst `text_out`, wat de lijst van karakterindices is. Gelukkig verwacht de functie `cross_entropy` niet-genormaliseerde netwerkoutput als eerste argument en het klasnummer als tweede, wat precies is wat we hebben. Het voert ook automatische averaging uit over de minibatch-grootte.

We beperken de training ook tot `samples_to_train` samples, zodat we niet te lang hoeven te wachten. We moedigen je aan om te experimenteren en langere trainingen te proberen, mogelijk voor meerdere epochs (in dat geval zou je een extra lus rond deze code moeten maken).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Dit voorbeeld genereert al behoorlijk goede tekst, maar er zijn verschillende manieren om het verder te verbeteren:

* **Betere minibatch-generatie**. De manier waarop we data voor training hebben voorbereid, was door één minibatch uit één sample te genereren. Dit is niet ideaal, omdat minibatches allemaal verschillende groottes hebben, en sommige zelfs niet gegenereerd kunnen worden omdat de tekst kleiner is dan `nchars`. Bovendien belasten kleine minibatches de GPU niet voldoende. Het zou verstandiger zijn om één groot tekstblok uit alle samples te halen, vervolgens alle input-outputparen te genereren, deze te schudden en minibatches van gelijke grootte te maken.

* **Meerdere lagen LSTM**. Het is zinvol om 2 of 3 lagen van LSTM-cellen te proberen. Zoals we in de vorige eenheid hebben besproken, haalt elke laag van een LSTM bepaalde patronen uit de tekst, en in het geval van een generator op tekenniveau kunnen we verwachten dat een lagere LSTM-laag verantwoordelijk is voor het herkennen van lettergrepen, terwijl hogere lagen zich richten op woorden en woordcombinaties. Dit kan eenvoudig worden geïmplementeerd door een parameter voor het aantal lagen door te geven aan de LSTM-constructor.

* Je kunt ook experimenteren met **GRU-eenheden** om te zien welke beter presteren, en met **verschillende groottes van verborgen lagen**. Een te grote verborgen laag kan resulteren in overfitting (bijvoorbeeld dat het netwerk de exacte tekst leert), terwijl een kleinere grootte mogelijk geen goed resultaat oplevert.


## Zachte tekstgeneratie en temperatuur

In de vorige definitie van `generate` kozen we altijd het karakter met de hoogste waarschijnlijkheid als het volgende karakter in de gegenereerde tekst. Dit resulteerde er vaak in dat de tekst "cyclisch" werd en steeds dezelfde karakterreeksen herhaalde, zoals in dit voorbeeld:
```
today of the second the company and a second the company ...
```

Als we echter naar de waarschijnlijkheidsverdeling voor het volgende karakter kijken, kan het zijn dat het verschil tussen een paar hoogste waarschijnlijkheden niet groot is, bijvoorbeeld: één karakter kan een waarschijnlijkheid van 0,2 hebben, en een ander 0,19, enzovoort. Bijvoorbeeld, bij het zoeken naar het volgende karakter in de reeks '*play*', kan het volgende karakter net zo goed een spatie zijn, of **e** (zoals in het woord *player*).

Dit leidt ons tot de conclusie dat het niet altijd "eerlijk" is om het karakter met de hoogste waarschijnlijkheid te kiezen, omdat het kiezen van het op één na hoogste karakter ook tot betekenisvolle tekst kan leiden. Het is verstandiger om **karakters te bemonsteren** uit de waarschijnlijkheidsverdeling die door de netwerkoutput wordt gegeven.

Dit bemonsteren kan worden gedaan met behulp van de functie `multinomial`, die de zogenaamde **multinomiale verdeling** implementeert. Een functie die deze **zachte** tekstgeneratie implementeert, is hieronder gedefinieerd:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

We hebben een extra parameter geïntroduceerd genaamd **temperatuur**, die wordt gebruikt om aan te geven hoe strikt we ons moeten houden aan de hoogste waarschijnlijkheid. Als de temperatuur 1.0 is, doen we eerlijke multinomiale steekproeven, en wanneer de temperatuur naar oneindig gaat - worden alle waarschijnlijkheden gelijk, en selecteren we willekeurig het volgende karakter. In het onderstaande voorbeeld kunnen we zien dat de tekst betekenisloos wordt wanneer we de temperatuur te veel verhogen, en het lijkt op "gecyclet" hard gegenereerde tekst wanneer het dichter bij 0 komt.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
